In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
model_name = "croissantllm/CroissantLLMChat-v0.1"

In [ ]:
from pathlib import Path

In [ ]:
model_repository = Path.cwd().joinpath("models")

In [ ]:
model_repository.exists()

In [ ]:
croissant_path = model_repository.joinpath(model_name)

In [ ]:
snapshot_download(repo_id=model_name, local_dir=croissant_path,
                  local_dir_use_symlinks=False, revision="main")

After downloading the model, we need to save it to gguf file, which is the file format used by llam cpp

In [ ]:
croissant_path_gguf_path = croissant_path.parent.joinpath(f"{model_name.split('/')[0]}_32.gguf")
assert croissant_path_gguf_path.parent.exists()
croissant_path_gguf_16_bits = croissant_path_gguf_path.with_suffix(".16bits.gguf")
croissant_path_gguf_8_bits = croissant_path_gguf_path.with_suffix(".8bits.gguf")
croissant_path_gguf_path.unlink()
croissant_path_gguf_16_bits.unlink()
croissant_path_gguf_8_bits.unlink()
croissant_path_gguf_path = croissant_path_gguf_path.__str__()

In [ ]:
croissant_path_gguf_path_8_bits = croissant_path_gguf_8_bits.__str__()
croissant_path_gguf_path_16_bits = croissant_path_gguf_16_bits.__str__()

In [ ]:
croissant_path_gguf_path_8_bits

In [ ]:
llama_cpp_path = Path.cwd().parent.joinpath("llama.cpp")
convert_script_path = llama_cpp_path.joinpath(
    "convert_hf_to_gguf.py").__str__()
legacy_convert_script_path = llama_cpp_path.joinpath("examples",
    "convert_legacy_llama.py")

In [ ]:
model_path =croissant_path.__str__()

####  Start with the tokenizer

In [ ]:
tokenizer_model_path = croissant_path.joinpath("tokenizer.model").__str__()

In [ ]:
from transformers import AutoTokenizer

In [ ]:
AutoTokenizer.from_pretrained(
    model_path,
    cache_dir=model_path,
    local_files_only=True,
)

In [ ]:
!python $convert_script_path $model_path --outfile $croissant_path_gguf_path --outtype f32

In [ ]:
!python $convert_script_path $model_path --outfile $croissant_path_gguf_path_8_bits --outtype f32

In [ ]:
!python $convert_script_path $model_path --model-name $model_name --outfile $croissant_path_gguf_path_16_bits --outtype f16

In [ ]:
tokenizer_model_path = croissant_path.parent.joinpath("tokenizer.model")

In [ ]:
!python $convert_script_path $model_path --vocab-only  --outfile $tokenizer_model_path --outtype f32